GPT2 Classifier ... fail

In [25]:
def zero_shot_with_finetuning():
    from ProgressPrinter import ProgressPrinter
    from GPT2 import GPT2Classifier
    from PersonalizedCitation import train_loader
    import torch

    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=2)
    zeroshot = GPT2Classifier(train.num_labels).to(device)

    with ProgressPrinter('loss', '0 shot acc') as printer:
        for inputs, profiles, answers in train:
            with torch.no_grad():
                target = torch.Tensor([ int(answer == train.choices[1]) for answer in answers ]).long().to(device)
                zeroshotacc = (zeroshot.predict(inputs).argmax(dim=1) == target).float().mean().item()
                
            loss = zeroshot.learn(inputs, target)
            printer.addobs(loss, zeroshotacc)

from Fork import SubProcess
with SubProcess() as process: process.parent or zero_shot_with_finetuning()

n                loss      since 0 shot acc      since     dt (s)
1              0.0607     0.0607          1          1      0.693
2                6.76       13.5        0.5          0      0.901
4                6.54       6.31      0.625       0.75       1.22
8                5.29       4.04      0.562        0.5       1.86
16               3.75       2.21      0.469      0.375       3.14
32               2.28      0.817        0.5      0.531       5.71
64                1.5      0.722      0.523      0.547       10.8
128              1.11      0.719      0.488      0.453       21.1
256             0.907      0.704       0.49      0.492       41.7
512             0.803      0.699      0.497      0.504       82.9
1024             0.75      0.697       0.51      0.522        166
2048            0.723      0.696      0.511      0.512        333
4096            0.709      0.696      0.502      0.494        664
4841            0.707      0.695      0.504      0.514        784


In [29]:
def one_shot_with_finetuning():
    from ProgressPrinter import ProgressPrinter
    from GPT2 import GPT2Classifier
    from PersonalizedCitation import train_loader
    from ReplayWrapper import ReplayWrapper
    import torch

    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=2)
    fewshot = ReplayWrapper(GPT2Classifier(train.num_labels).to(device), replay_count=4, nbatches=128)

    with ProgressPrinter('1 loss', '1 shot acc') as printer:
        for inputs, profiles, answers in train:
            with torch.no_grad():
                target = torch.Tensor([ int(answer == train.choices[1]) for answer in answers ]).long().to(device)

                shots = []
                for input_embedding, profile in zip(train.embed(inputs), profiles):
                    profile_embeddings = train.embeddings(profile)
                    indices = torch.topk(input_embedding @ profile_embeddings.T, dim=0, k=1).indices.to('cpu')
                    shots.append(train.stringify_articles(indices))

                fewinputs = [ shot[0] + '\n\n' + input for input, shot in zip(inputs, shots) ]
                fewshotacc = (fewshot.predict(inputs).argmax(dim=1) == target).float().mean().item()
                
            fewloss = fewshot.learn(fewinputs, target)
            
            printer.addobs(fewloss, fewshotacc)

from Fork import SubProcess
with SubProcess() as process: process.parent or one_shot_with_finetuning()

n              1 loss      since 1 shot acc      since     dt (s)
1              0.0708     0.0708          1          1       1.53
2                3.11       6.15        0.5          0       2.17
4                4.88       6.66      0.375       0.25       3.62
8                3.05       1.21      0.438        0.5       5.95
16               2.14       1.23      0.469        0.5       10.7
32               1.44      0.732      0.531      0.594       20.5
64               1.08      0.718      0.547      0.562       40.6
128             0.898       0.72      0.504      0.461       80.3
256             0.803      0.707      0.512       0.52        285
512             0.802        0.8       0.52      0.527        766
1024            0.887      0.972      0.516      0.513    1.8e+03
2048            0.951       1.01      0.512      0.509   4.79e+03
4096            0.978          1      0.503      0.494   9.22e+03
4841            0.979      0.988      0.507      0.529   1.04e+04
